In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [3]:
inplace1 = (train_df["full_sq"] < train_df["life_sq"]) & (train_df["full_sq"] < 10)
temp3 = train_df["full_sq"].copy()
temp3[inplace1] = train_df["life_sq"][inplace1]
train_df["full_sq"] = temp3

In [4]:
inplace2 = train_df["life_sq"] < 2
temp4 = train_df["life_sq"].copy()
temp4[inplace2] = train_df["full_sq"][inplace2]
train_df["life_sq"] = temp4

In [5]:
inplace3 = (train_df["full_sq"] < train_df["life_sq"]) & (train_df["life_sq"] > 30)
temp5 = train_df["life_sq"].copy()
temp5[inplace3] = train_df["full_sq"][inplace3]
train_df["life_sq"] = temp5

In [6]:
inplace1 = (test_df["full_sq"] < test_df["life_sq"]) & (test_df["full_sq"] < 10)
temp3 = test_df["full_sq"].copy()
temp3[inplace1] = test_df["life_sq"][inplace1]
test_df["full_sq"] = temp3
inplace2 = test_df["life_sq"] < 2
temp4 = test_df["life_sq"].copy()
temp4[inplace2] = test_df["full_sq"][inplace2]
test_df["life_sq"] = temp4
inplace3 = (test_df["full_sq"] < test_df["life_sq"]) & (test_df["life_sq"] > 30)
temp5 = test_df["life_sq"].copy()
temp5[inplace3] = test_df["full_sq"][inplace3]
test_df["life_sq"] = temp5

In [7]:
train_df = train_df.replace({"no": -1, "yes": 1})
test_df = test_df.replace({"no": -1, "yes": 1})

In [8]:
train_df.drop(['product_type'], axis=1, inplace=True)
test_df.drop(['product_type'], axis=1, inplace=True)
train_df.drop(['ecology'], axis=1, inplace=True)
test_df.drop(['ecology'], axis=1, inplace=True)

In [9]:
y_train = train_df['price_doc'].values
id_test = test_df['id']
train_df.drop(['timestamp'], axis=1, inplace=True)
test_df.drop(['timestamp'], axis=1, inplace=True)
train_df.drop("id", axis=1, inplace=True)
train_df.drop("price_doc", axis=1, inplace=True)
test_df.drop("id", axis=1, inplace=True)
train_df.drop("sub_area", axis=1, inplace=True)
test_df.drop("sub_area", axis=1, inplace=True)

In [10]:
train_df["material"].ix[train_df["material"] == 33] = 3

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':
C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'min_child_weight':1,
    'silent': 1,
    'seed':0
}

dtrain = xgb.DMatrix(train_df, y_train)
dtest = xgb.DMatrix(test_df)

In [12]:
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= 450)

In [13]:
y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output["price_doc"] = output["price_doc"] * 0.96
output.head()

,id,price_doc
0,30474,5348669.0
1,30475,7877209.0
2,30476,5186856.5
3,30477,5527826.5
4,30478,4993525.0


In [14]:
output.to_csv('NewStart3.csv', index=False)